In [10]:
!pip install python-dotenv

In [11]:
# ================================
# Adzuna Job Fetcher (Colab / GitHub Ready)
# ================================

import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from dotenv import load_dotenv

# -------------------------
# Load environment variables
# -------------------------
load_dotenv()  # local 用 .env；Colab 沒有也不會壞

APP_ID = os.getenv("ADZUNA_APP_ID")
APP_KEY = os.getenv("ADZUNA_APP_KEY")

if not APP_ID or not APP_KEY:
    raise ValueError("❌ Missing Adzuna API credentials")

# -------------------------
# Config
# -------------------------
COUNTRY = "us"
CITY = "Baltimore"
KEYWORDS = ["data scientist", "data analyst"]

RESULTS_PER_PAGE = 50
MAX_PAGES = 20
DAYS_BACK = 60

BASE_URL = f"https://api.adzuna.com/v1/api/jobs/{COUNTRY}/search"

# -------------------------
# 🎯 Title Filtering Rules
# -------------------------
TARGET_TITLE_KEYWORDS = [
    "data scientist",
    "senior data scientist",
    "sr. data scientist",
    "lead data scientist",
    "principal data scientist",
    "staff data scientist",
    "data analyst",
    "senior data analyst",
]

EXCLUDED_TITLE_KEYWORDS = [
    "architect",
    "engineer",
    "manager",
    "consultant",
    "director",
    "vp",
    "vice president",
    "head of",
    "marketing",
    "sales",
    "product",
]

def is_relevant_title(title: str) -> bool:
    if not title:
        return False

    title_lower = title.lower()

    has_target = any(k in title_lower for k in TARGET_TITLE_KEYWORDS)
    has_excluded = any(k in title_lower for k in EXCLUDED_TITLE_KEYWORDS)

    return has_target and not has_excluded

# -------------------------
# Fetch Jobs
# -------------------------
all_jobs = []
cutoff_date = datetime.now() - timedelta(days=DAYS_BACK)

for keyword in KEYWORDS:
    for page in range(1, MAX_PAGES + 1):

        params = {
            "app_id": APP_ID,
            "app_key": APP_KEY,
            "what": keyword,
            "where": CITY,
            "results_per_page": RESULTS_PER_PAGE,
            "sort_by": "date",
        }

        print(f"📥 Fetching '{keyword}' | page {page}")

        response = requests.get(f"{BASE_URL}/{page}", params=params)
        response.raise_for_status()

        data = response.json()
        results = data.get("results", [])

        if not results:
            break

        for job in results:
            created = job.get("created")
            created_dt = datetime.fromisoformat(created.replace("Z", ""))

            if created_dt < cutoff_date:
                continue

            title = job.get("title")

            if not is_relevant_title(title):
                continue

            all_jobs.append({
                "title": title,
                "company": job.get("company", {}).get("display_name"),
                "location": job.get("location", {}).get("display_name"),
                "created": created,
                "description": job.get("description"),
                "redirect_url": job.get("redirect_url"),
                "keyword": keyword,
            })

print("\n✅ Fetch completed")
print(f"🔢 Total jobs collected (after filtering): {len(all_jobs)}")

# -------------------------
# Save to CSV
# -------------------------
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

output_path = data_dir / "jobs_filtered.csv"
df = pd.DataFrame(all_jobs)
df.to_csv(output_path, index=False)

print(f"💾 Saved to {output_path.resolve()}")

df.head()


📥 Fetching 'data scientist' | page 1
📥 Fetching 'data scientist' | page 2
📥 Fetching 'data scientist' | page 3
📥 Fetching 'data analyst' | page 1
📥 Fetching 'data analyst' | page 2
📥 Fetching 'data analyst' | page 3

✅ Fetch completed
🔢 Total jobs collected (after filtering): 22
💾 Saved to /content/data/jobs_filtered.csv


,title,company,location,created,description,redirect_url,keyword
0,Data Scientist,Peraton,"Linthicum Heights, Anne Arundel County",2026-01-08T14:03:10Z,Peraton is hiring a Data Scientist to join our...,https://www.adzuna.com/details/5574350423?utm_...,data scientist
1,Data Scientist with Security Clearance,Peraton,"Linthicum Heights, Anne Arundel County",2026-01-07T23:42:44Z,About Peraton Peraton is a next-generation nat...,https://www.adzuna.com/land/ad/5573122190?se=4...,data scientist
2,Sr. Data Scientist - Clinical AI,Johns Hopkins Medicine,"East Case, Baltimore",2026-01-07T19:30:42Z,Monday-Friday 8:30am-5pm Hybrid/Remote *On-Sit...,https://www.adzuna.com/details/5572834821?utm_...,data scientist
3,"Lead Data Scientist, Strategic Analytics - Dat...",Deloitte,"Druid, Baltimore",2026-01-07T15:29:00Z,Are you passionate about harnessing data to dr...,https://www.adzuna.com/land/ad/5572600425?se=4...,data scientist
4,Data Scientist Fellow,Baltimore Orioles,"Baltimore, Baltimore County",2025-12-24T17:31:02Z,Title: Data Scientist Fellow Reports To: Assis...,https://www.adzuna.com/details/5554893511?utm_...,data scientist


In [7]:
# ================================
# Skill Extraction (Colab Version)
# ================================

import pandas as pd
import re
from collections import Counter
from pathlib import Path

# -------------------------
# Load job data
# -------------------------
input_path = Path("data/jobs_filtered.csv")
df = pd.read_csv(input_path)

print(f"📄 Loaded {len(df)} job postings")

# -------------------------
# Skill dictionary (Data roles)
# -------------------------
SKILLS = {
    "Python": ["python"],
    "SQL": ["sql", "postgres", "mysql", "sqlite"],
    "R": [" r ", " r,", " r.", " r/"],
    "Excel": ["excel"],
    "Tableau": ["tableau"],
    "Power BI": ["power bi", "powerbi"],
    "Pandas": ["pandas"],
    "NumPy": ["numpy"],
    "Scikit-learn": ["scikit-learn", "sklearn"],
    "TensorFlow": ["tensorflow"],
    "PyTorch": ["pytorch"],
    "Statistics": ["statistics", "statistical"],
    "Machine Learning": ["machine learning", "ml"],
    "Deep Learning": ["deep learning"],
    "NLP": ["nlp", "natural language processing"],
    "AWS": ["aws", "amazon web services"],
    "GCP": ["gcp", "google cloud"],
    "Azure": ["azure"],
    "Spark": ["spark", "pyspark"],
    "Hadoop": ["hadoop"],
    "Docker": ["docker"],
    "Kubernetes": ["kubernetes", "k8s"],
    "Git": ["git", "github"],
    "Linux": ["linux"],
    "ETL": ["etl", "data pipeline"],
    "Big Data": ["big data"]
}

# -------------------------
# Skill extraction logic
# -------------------------
skill_counter = Counter()

for desc in df["description"].dropna():
    text = desc.lower()
    found_skills = set()

    for skill, keywords in SKILLS.items():
        for kw in keywords:
            if re.search(rf"\b{re.escape(kw)}\b", text):
                found_skills.add(skill)
                break

    for skill in found_skills:
        skill_counter[skill] += 1

# -------------------------
# Convert to DataFrame
# -------------------------
skill_df = (
    pd.DataFrame(skill_counter.items(), columns=["skill", "count"])
    .sort_values(by="count", ascending=False)
    .reset_index(drop=True)
)

# -------------------------
# Save results
# -------------------------
output_path = Path("data/skill_frequency.csv")
skill_df.to_csv(output_path, index=False)

print(f"✅ Skill frequency saved to {output_path.resolve()}")

skill_df.head(10)


📄 Loaded 21 job postings
✅ Skill frequency saved to /content/data/skill_frequency.csv


,skill,count
0,Machine Learning,3
1,Statistics,2
2,Big Data,1
3,ETL,1
